# 基于langchain创建自己专属的对话大模型

1. 领域精准回答

2. 数据更新频繁

3. 生成内容可解释追溯

4. 数据隐私保护

通过这个例子，我们将基于`LangChain`, `OpenAI(LLM)`,`vector DB` 构建一个属于自己的LLM模型

主要使用技术——————__*Retrieval Augmented Generation*__


## 环境准备

可以选择官方的openai_api_key

也可以选择中间代理商，此时需要openai_api_base

In [6]:
! pip install -qU \
    langchain==0.0.316 \
    openai==0.28.1  \
    tiktoken==0.12.0  \
    cohere \
    chromadb==0.4.15


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 创建一个对话模型(No RAG)

In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from getpass import getpass

OPENAI_API_KEY = "***********"

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

chat = ChatOpenAI(
    openai_api_base="https://**********/api/v1",
    openai_api_key=os.environ['OPENAI_API_KEY'],
    model="openai/gpt-3.5-turbo"
)

In [8]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Knock knock."),
    AIMessage(content="Who's there?"),
    HumanMessage(content="Orange"),

]

In [9]:
res = chat(messages)
print(res.content)



Orange who? 

Orange you glad I didn't say banana? 😄



Want to play again?


In [10]:
messages.append(res)
res = chat(messages)
print(res.content)

Orange you glad I'm here to assist you?


### 处理LLM存在的缺陷

1. 容易出现幻觉

2. 信息滞后

3. 专业领域深度知识匮乏

In [11]:
messages = [
    SystemMessage(content="你是一个专业的知识助手。"),
    HumanMessage(content="你知道baichuan2模型吗？"),
]

res = chat(messages)
print(res.content)

抱歉，我不熟悉"baichuan2"模型。请问你能提供更多背景信息或者上下文吗？这样我才能更好地帮助你。


In [12]:
baichuan2_information = [
    "Baichuan 2是一个大规模多语言语言模型，它专注于训练在多种语言中表现优异的模型，包括不仅限于英文。这使得Baichuan 2在处理各种语言的任务时能够取得显著的性能提升。",
    "Baichuan 2是从头开始训练的，使用了包括了2.6万亿个标记的庞大训练数据集。相对于以往的模型，Baichuan 2提供了更丰富的数据资源，从而能够更好地支持多语言的开发和应用。",
    "Baichuan 2不仅在通用任务上表现出色，还在特定领域（如医学和法律）的任务中展现了卓越的性能。这为特定领域的应用提供了强有力的支持。"
]

source_knowledge = "\n".join(baichuan2_information)
print(source_knowledge)

Baichuan 2是一个大规模多语言语言模型，它专注于训练在多种语言中表现优异的模型，包括不仅限于英文。这使得Baichuan 2在处理各种语言的任务时能够取得显著的性能提升。
Baichuan 2是从头开始训练的，使用了包括了2.6万亿个标记的庞大训练数据集。相对于以往的模型，Baichuan 2提供了更丰富的数据资源，从而能够更好地支持多语言的开发和应用。
Baichuan 2不仅在通用任务上表现出色，还在特定领域（如医学和法律）的任务中展现了卓越的性能。这为特定领域的应用提供了强有力的支持。


In [13]:
query = "你知道baichuan2模型吗？"

prompt_template = f"""基于以下内容回答问题：

内容:
{source_knowledge}

Query: {query}"""

In [14]:
prompt = HumanMessage(
    content=prompt_template
)

messages.append(prompt)

res = chat(messages)

print(res.content)

Baichuan 2是一个大规模多语言语言模型，它专注于训练在多种语言中表现优异的模型，包括不仅限于英文。这使得Baichuan 2在处理各种语言的任务时能够取得显著的性能提升。Baichuan 2是从头开始训练的，使用了包括了2.6万亿个标记的庞大训练数据集。相对于以往的模型，Baichuan 2提供了更丰富的数据资源，从而能够更好地支持多语言的开发和应用。Baichuan 2不仅在通用任务上表现出色，还在特定领域（如医学和法律）的任务中展现了卓越的性能。这为特定颀域的应用提供了强有力的支持。


## 创建一个RAG对话模型

### 1. 加载数据

In [11]:
! pip install pypdf

Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("2309.10305v2.pdf")

pages = loader.load_and_split()

In [17]:
pages[0]

Document(page_content='Baichuan 2: Open Large-scale Language Models\nAiyuan Yang, Bin Xiao, Bingning Wang, Borong Zhang, Chao Yin, Chenxu Lv, Da Pan\nDian Wang, Dong Yan, Fan Yang, Fei Deng, Feng Wang, Feng Liu, Guangwei Ai\nGuosheng Dong, Haizhou Zhao, Hang Xu, Haoze Sun, Hongda Zhang, Hui Liu, Jiaming Ji\nJian Xie, Juntao Dai, Kun Fang, Lei Su, Liang Song, Lifeng Liu, Liyun Ru, Luyao Ma\nMang Wang, Mickel Liu, MingAn Lin, Nuolan Nie, Peidong Guo, Ruiyang Sun\nTao Zhang, Tianpeng Li, Tianyu Li, Wei Cheng, Weipeng Chen, Xiangrong Zeng\nXiaochuan Wang, Xiaoxi Chen, Xin Men, Xin Yu, Xuehai Pan, Yanjun Shen, Yiding Wang\nYiyu Li, Youxin Jiang, Yuchen Gao, Yupeng Zhang, Zenan Zhou, Zhiying Wu\nBaichuan Inc.\nAbstract\nLarge language models (LLMs) have\ndemonstrated remarkable performance on\na variety of natural language tasks based\non just a few examples of natural language\ninstructions, reducing the need for extensive\nfeature engineering. However, most powerful\nLLMs are closed-source

### 2. 知识切片，将文档分割成均匀的块。每个块是一段原始文本

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)

docs = text_splitter.split_documents(pages)

In [19]:
len(docs)

216

### 3. 利用embedding模型对每个文本片进行向量化，并存储到向量数据库中

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embed_model = OpenAIEmbeddings(
    openai_api_base="https://*********/api/v1",
    openai_api_key=OPENAI_API_KEY,
    model="openai/text-embedding-3-large"
)
vectorstore = Chroma.from_documents(
    documents=docs, 
    embedding=embed_model, 
    collection_name="openai_embed"
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


### 4. 通过向量相似度检索和问题最相关的k个文档

In [27]:
query = "How large is the baichuan2 vocabulary?"
result = vectorstore.similarity_search(query ,k = 2)
print(result)

[Document(page_content='adequate training of each word embedding. We\nhave taken both these aspects into account. We\nhave expanded the vocabulary size from 64,000\nin Baichuan 1 to 125,696, aiming to strike a\nbalance between computational efficiency and\nmodel performance.\nTokenizer V ocab Size Compression Rate ↓\nLLaMA 2 32,000 1.037\nBloom 250,680 0.501\nChatGLM 2 64,794 0.527\nBaichuan 1 64,000 0.570\nBaichuan 2 125,696 0.498\nTable 2: The vocab size and text compression rate of', metadata={'page': 2, 'source': '2309.10305v2.pdf'}), Document(page_content='adequate training of each word embedding. We\nhave taken both these aspects into account. We\nhave expanded the vocabulary size from 64,000\nin Baichuan 1 to 125,696, aiming to strike a\nbalance between computational efficiency and\nmodel performance.\nTokenizer V ocab Size Compression Rate ↓\nLLaMA 2 32,000 1.037\nBloom 250,680 0.501\nChatGLM 2 64,794 0.527\nBaichuan 1 64,000 0.570\nBaichuan 2 125,696 0.498\nTable 2: The vocab 

### 5. 原始`query`与检索得到的文本组合起来输入到语言模型，得到最终的回答

In [29]:
def augment_prompt(query: str):
  # 获取top3的文本片段
  results = vectorstore.similarity_search(query, k=3)
  source_knowledge = "\n".join([x.page_content for x in results])
  # 构建prompt
  augmented_prompt = f"""Using the contexts below, answer the query.

  contexts:
  {source_knowledge}

  query: {query}"""
  return augmented_prompt

print(augment_prompt(query))

Using the contexts below, answer the query.

  contexts:
  adequate training of each word embedding. We
have taken both these aspects into account. We
have expanded the vocabulary size from 64,000
in Baichuan 1 to 125,696, aiming to strike a
balance between computational efficiency and
model performance.
Tokenizer V ocab Size Compression Rate ↓
LLaMA 2 32,000 1.037
Bloom 250,680 0.501
ChatGLM 2 64,794 0.527
Baichuan 1 64,000 0.570
Baichuan 2 125,696 0.498
Table 2: The vocab size and text compression rate of
adequate training of each word embedding. We
have taken both these aspects into account. We
have expanded the vocabulary size from 64,000
in Baichuan 1 to 125,696, aiming to strike a
balance between computational efficiency and
model performance.
Tokenizer V ocab Size Compression Rate ↓
LLaMA 2 32,000 1.037
Bloom 250,680 0.501
ChatGLM 2 64,794 0.527
Baichuan 1 64,000 0.570
Baichuan 2 125,696 0.498
Table 2: The vocab size and text compression rate of
languages, such as Chinese.
In th

In [30]:
# 创建prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)

messages.append(prompt)

res = chat(messages)

print(res.content)

Baichuan 2 has a vocabulary size of 125,696 words.
